In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

import matplotlib.pyplot as plt

In [ ]:
#stocks_dir = '/Volumes/Photos/stocks/'
stocks_dir = './'

symbols = ['UNH', 'AAPL', 'GOOG', 'TSLA', 'TWTR']

stocks = {}  # key (str) - symbol, value - DataFrame

roll_win = 10  # rolling window (days) for mean prices
for s in symbols:
    file_name = stocks_dir + s + '.csv'
    df = pd.read_csv(file_name, index_col='Date', parse_dates=True, usecols=['Date', 'Adj. Close'])
    df.columns = ['price']
    df['price'] = df['price'].rolling(roll_win, center=True, min_periods=1).mean()
    stocks[s] = df['price']


sp500 = pd.read_csv('SP500_yahoo.csv', index_col='Date', parse_dates=True, na_values=['.'])
sp500 = sp500[['Adj Close']]
sp500.columns = ['price']
sp500['price'] = sp500['price'].rolling(roll_win, center=True, min_periods=1).mean()
sp500 = sp500['price']
sp500 = sp500[sp500.index.isin(stocks[symbols[0]].index)]

symbols.extend(['SPY'])
stocks['SPY']  = sp500

# Daily returns
stocks_ret = {k: pd.Series(v.pct_change().iloc[1:], name='daily_ret') for k, v in stocks.items()}

print("UNH prices")
print(stocks['UNH'].head(3))
print("\nUNH daily returns")
print(stocks_ret['UNH'].head(3))

In [ ]:
print(stocks['UNH'].shift().head(3))

### Pick asset and benchmark

In [ ]:
# Get data for the specified period and stocks
start, end = '2015-01-01', '2018-01-01'
asset_symbol, benchmark_symbol = 'UNH', 'SPY'

asset = stocks[asset_symbol].loc[start:end]
bench = stocks[benchmark_symbol].loc[start:end]

asset_ret = stocks_ret[asset_symbol].loc[start:end]
bench_ret = stocks_ret[benchmark_symbol].loc[start:end]

In [ ]:
print("asset_ret std: {}".format(asset_ret.std()))
print("bench_ret std: {}".format(bench_ret.std()))
print("asset_ret.corr(bench_ret): {}".format(asset_ret.corr(bench_ret)))
print("beta = (asset_ret.corr(bench_ret) ) * (asset_ret std)/ (bench_ret std): {}".format(asset_ret.corr(bench_ret) * asset_ret.std() / bench_ret.std()))

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(bench_ret, asset_ret)
beta, alpha = slope, intercept
print('beta: {}, alpha: {}'.format(slope, intercept))

In [ ]:
v1 = asset.iloc[-1] / asset.iloc[0]
v2 = bench.iloc[-1] / bench.iloc[0]
print('{} return: {}, bench return: {}'.format(asset_symbol, round(v1, 3), round(v2, 3)))

print('{} std: {}, bench std: {}'.format(asset_symbol, round(asset_ret.std(), 3), round(bench_ret.std(), 3)))

In [ ]:
plt.figure(figsize=(14, 14))
plt.scatter(bench_ret, asset_ret, alpha=0.5, s=50)
plt.grid()
plt.xlabel('SPY Ret')
plt.ylabel(asset_symbol + ' Ret')
lim = 0.08 / np.sqrt(roll_win)
plt.xlim([-lim, lim])
plt.ylim([-lim, lim])

# Fit Line
min_spy = bench_ret.values.min()
max_spy = bench_ret.values.max()

spy_line = np.linspace(min_spy,max_spy,100)
y = spy_line * beta + alpha

plt.plot(spy_line,y,'r');